In this notebook, we will see how we can preprocess the data that was downloaded from Open Neuro. 

The preproces we used 
- `filter` to filter the signals between desired Hz
- `resample` to resample the eeg signal from acqusition frequency to a desired frequency
- `ICA` to remove `ecg, eog` related artifacts 
- `fized length epochs` to break the continuous signal to number of samples

The following approaches were used to preproces that data

Approach 1

- filter between 1 and 40
- resample from 500hz to 100 hz
- remove artifacts based on ICA
- epoch 50s

Approach 2

- filter between 1 and 40
- resample from 500hz to 100 hz
- remove artifacts based on ICA
- epoch 50s and average

Approach 3

- filter between 1 and 20
- resample from 500hz to 100 hz
- remove artifacts based on ICA on epochs
- epoch 50s

Approach 4

- filter between 1 and 20
- resample from 500hz to 100 hz
- remove artifacts based on ICA on epochs
- epoch 50s and average


In [3]:
%pip install fastcore mne mne-bids PyQt5 -Uqq

Note: you may need to restart the kernel to use updated packages.


In [5]:
import os
import matplotlib
from pathlib import Path
import mne
import mne_bids
import numpy as np
from glob import glob
from fastcore.parallel import parallel
import pandas as pd
import pickle

In [6]:
from sklearn.base import TransformerMixin,BaseEstimator
from sklearn.preprocessing import StandardScaler

In [7]:
def prepare_approach12(fn):
    '''
    approach1
    filter - 1 and 40
    resample - 100hz
    ica - 20 components
    epoch - 50s

    approach2
    same but average all epochs to one   
    '''
    path = f"processeddata/individuals/afterica"
    sub = str(fn).split('/')[-1].split('_')[0]
    session = str(fn).split('/')[-1].split('_')[1]
    label = str(fn).split('/')[-1].split('_')[2]

    read_raw_bids(bids_path=fn, verbose=False)
    raw.load_data()
    raw = raw.resample(100).filter(l_freq=1, h_freq=40)

    ica = mne.preprocessing.ICA(n_components=20, 
                                random_state=0)
    ica.fit(raw)

    bad_idx_ecg, scores_ecg = ica.find_bads_ecg(raw, 'Fp1', threshold=2)
    bad_idx_eog, scores_eog = ica.find_bads_eog(raw, 'Fp1', threshold=2)
    
    ica.exclude = bad_idx_ecg + bad_idx_eog

    raw_after = ica.apply(raw, 
                          exclude=ica.exclude)

    epochs_after = mne.make_fixed_length_epochs(raw_after,  
                                                duration=50,  
                                                overlap=0)

    fn1 = f"{path}/{label}_{sub}_{session}_approach1.npy"
    fn2 = f"{path}/{label}_{sub}_{session}_approach2.npy"

    np.save(fn1, epochs_after.get_data().astype(np.float16))
    np.save(fn2, epochs_after.average().get_data().astype(np.float16))

In [8]:
def prepare_approach34(fn):
    '''
    approach1
    filter - 1 and 20
    resample - 100hz
    ica - 20 components
    epoch - 50s

    approach2
    same but average all epochs to one   
    '''
    path = f"processeddata/individuals/afterica"
    sub = str(fn).split('/')[-1].split('_')[0]
    session = str(fn).split('/')[-1].split('_')[1]
    label = str(fn).split('/')[-1].split('_')[2]

    raw = mne.io.read_raw_brainvision(fn, preload=True)
    raw = raw.resample(100).filter(l_freq=1, h_freq=20)

    ica = mne.preprocessing.ICA(n_components=20, 
                                random_state=0)
    ica.fit(raw)
    bad_idx_ecg, scores_ecg = ica.find_bads_ecg(raw, 'Fp1', threshold=2)
    bad_idx_eog, scores_eog = ica.find_bads_eog(raw, 'Fp1', threshold=2)
    ica.exclude = bad_idx_ecg + bad_idx_eog

    raw_after = ica.apply(raw, 
                          exclude=ica.exclude)

    epochs_after = mne.make_fixed_length_epochs(raw_after,  
                                                duration=50,  
                                                overlap=0)

    fn1 = f"{path}/{label}_{sub}_{session}_approach3.npy"
    fn2 = f"{path}/{label}_{sub}_{session}_approach4.npy"

    np.save(fn1, epochs_after.get_data().astype(np.float16))
    np.save(fn2, epochs_after.average().get_data().astype(np.float16))

In [18]:
from mne.datasets import sample
import os.path as op

dataset = 'ds003685'
bids_root = op.join(op.dirname(sample.data_path()), dataset)

In [20]:
from mne_bids import BIDSPath, read_raw_bids, print_dir_tree, make_report

print(bids_root)
datatype = 'eeg'
suffix = 'eeg'
bids_paths = [ 
    BIDSPath(task='eyes open', suffix=suffix, datatype=datatype, root=bids_root),
    BIDSPath(task='eyes closed', suffix=suffix, datatype=datatype, root=bids_root)]

# filenames = []
# for path in Path('hackdataset').rglob('*.vhdr'):
#     filenames.append(path)

/Users/ben/mne_data/ds003685


In [21]:
from tqdm import tqdm

def get_vhdr_files(bids_path):
    # get names of the .vhdr files
    basenames = []
    match = bids_path.match()
    for i in tqdm(range(0, len(match))):
        if match[i].basename[-4:] == 'vhdr':
            basenames.append(match[i])
    return basenames

vhdr_files = []
for bids_path in bids_paths:
    vhdr_files.append(get_vhdr_files(bids_path))

100%|██████████| 9/9 [00:00<00:00, 61181.10it/s]


In [22]:
def get_raw_files(basenames):
    raws = []
    for basename in tqdm(basenames):
        raw = read_raw_bids(bids_path=basename, verbose=False)
        raw.load_data()
        raws.append(raw)
    
#print(vhdr_files)
open_eyes = get_raw_files(vhdr_files[0])
closed_eyes = get_raw_files(vhdr_files[1])

  0%|          | 0/3 [00:00<?, ?it/s]

Reading 0 ... 149999  =      0.000 ...   299.998 secs...


 33%|███▎      | 1/3 [00:00<00:00,  2.62it/s]

Reading 0 ... 149999  =      0.000 ...   299.998 secs...


 67%|██████▋   | 2/3 [00:00<00:00,  3.79it/s]

...

 33%|███▎      | 1/3 [00:00<00:00,  5.58it/s]

Reading 0 ... 149999  =      0.000 ...   299.998 secs...


 67%|██████▋   | 2/3 [00:00<00:00,  5.63it/s]

Reading 0 ... 149999  =      0.000 ...   299.998 secs...


100%|██████████| 3/3 [00:00<00:00,  5.73it/s]


In [10]:
%%capture
# for fn in open_eyes:
#   print(x)
parallel(prepare_approach12, 
         open_eyes, 
         n_workers=8, 
         progress=True)

In [11]:
%%capture
parallel(prepare_approach34, 
         open_eyes, 
         n_workers=8, 
         progress=True)

# Prepare df and Standard Scalar

In [12]:
class SScaler3D(BaseEstimator,TransformerMixin):

    def __init__(self):
        self.scaler = StandardScaler()

    def fit(self,X,y=None):
        self.scaler.fit(X.reshape(X.shape[0], -1))
        return self

    def transform(self,X):
        return self.scaler.transform(X.reshape(X.shape[0], -1)).reshape(X.shape)

In [13]:
def prepare_df_ss(approach):
    fns = glob(f'processeddata/individuals/afterica/*{approach}.npy')

    fns_2 = []
    label = []
    subject = []
    session = []
    for fn in fns:
        fns_2.append(fn)
        label.append(str(fn).split('/')[-1].split('_')[0])
        subject.append(str(fn).split('/')[-1].split('_')[1])
        session.append(str(fn).split('/')[-1].split('_')[2])

    df = pd.DataFrame([fns_2, label, subject, session]).T
    df.columns = ['fns', 'label', 'subject', 'session']

    val_subs = ['sub-51', 'sub-52', 'sub-53', 'sub-54', 'sub-55', 'sub-56', 'sub-57', 'sub-58', 'sub-59', 'sub-60']
    df['is_valid'] = False
    df.loc[df[df['subject'].isin(val_subs)].index, 'is_valid'] = True

    df.to_csv(f'{approach}infos.csv', index=False)

    if approach in ['approach1', 'approach3']:
        features = []
        for fn in df[df['is_valid']==False].fns.values:
            if np.load(fn).shape[0] == 6:
                features.append(np.load(fn))
        
        features = np.stack(features, 0)
        ss = SScaler3D()
        ss.fit(features)

        pickle.dump(ss, open(f'ss_{approach}.pkl', 'wb'))

    else:
      features = []
      for fn in df[df['is_valid']==False].fns.values:
          features.append(np.load(fn))
      
      features = np.stack(features, 0)
      ss = StandardScaler()
      ss.fit(features)

      pickle.dump(ss, open(f'ss_{approach}.pkl', 'wb'))

In [14]:
fns = glob(f'processeddata/individuals/afterica/*approach1.npy')

fns_2 = []
label = []
subject = []
session = []
for fn in fns:
    fns_2.append(fn)
    label.append(str(fn).split('/')[-1].split('_')[0])
    subject.append(str(fn).split('/')[-1].split('_')[1])
    session.append(str(fn).split('/')[-1].split('_')[2])

df = pd.DataFrame([fns_2, label, subject, session]).T
df.columns = ['fns', 'label', 'subject', 'session']

val_subs = ['sub-51', 'sub-52', 'sub-53', 'sub-54', 'sub-55', 'sub-56', 'sub-57', 'sub-58', 'sub-59', 'sub-60']
df['is_valid'] = False
df.loc[df[df['subject'].isin(val_subs)].index, 'is_valid'] = True

df.to_csv(f'approach1infos.csv', index=False)


# features = np.stack(features, 0)
# if approach in ['approach1', 'approach3']:
#     ss = SScaler3D()
#     ss.fit(features)

#     pickle.dump(ss, open(f'ss_{approach}.pkl', 'wb'))

# else:
#   ss = StandardScaler()
#   ss.fit(features)

#   pickle.dump(ss, open(f'ss_{approach}.pkl', 'wb'))

In [15]:
features = []
for fn in df[df['is_valid']==False].fns.values:
    if np.load(fn).shape[0] == 6:
        features.append(np.load(fn))


In [16]:
features = np.stack(features, 0)

ValueError: need at least one array to stack

In [ ]:
prepare_df_ss('approach1')

In [ ]:
prepare_df_ss('approach2')

In [ ]:
prepare_df_ss('approach3')

In [ ]:
prepare_df_ss('approach4')

In [ ]:
df

In [ ]:
features.shape